In [26]:
import numpy as np
import pandas as pd
import json
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import VotingRegressor, AdaBoostRegressor

In [2]:
df = pd.read_csv('extracted_dataset.csv')
df.head()

,Course Code,Course Title,Type,Course Year,Student Number,Department Code,Course Level,GPA,Standing,Completed Credits,...,GPA Student - Subject,Avg. Grade - Taken,Avg. Grade - Students_Subject,Letter Grade,Status,Semester,Theoritical,Practical,Course Credit,ECTS
0,UNI 113,Textual Analysis and Effective Com. I,NaN,1,351,IE,Undergraduate,4.05,Freshman,18,...,4.050000,2.120833,2.703226,A+,Successful,2010 - Fall,3,0,3,5
1,MGT 553,Yöneticiler İçin Hukuk,NaN,5,1181,MBA/NT,Graduate,3.38,Freshman,15,...,3.400000,3.228571,3.353425,B+,Successful,2010 - Fall,3,0,3,5
2,EM 475,Stratejik Yönetim,NaN,4,1227,MBA/NT,Graduate,3.00,Freshman,15,...,3.300000,3.227273,3.227273,B+,Successful,2010 - Fall,3,0,3,5
3,MGT 511,Küresel Yönetim Düşüncesi,NaN,5,1182,MBA/NT,Graduate,3.54,Freshman,15,...,3.500000,3.576471,3.353425,A-,Successful,2010 - Fall,3,0,3,5
4,UNI 101,Mathematical Reasoning,NaN,1,344,EE,Undergraduate,4.03,Freshman,18,...,4.033333,2.144444,2.703226,A,Successful,2010 - Fall,3,0,3,5


In [3]:
grad_courses = []
for idx in df.index:
    if df.iloc[idx, 3] > 4:
        grad_courses.append(idx)
for row_index in grad_courses:
    df.drop(row_index, inplace=True)
df.index = range(len(df))
df

,Course Code,Course Title,Type,Course Year,Student Number,Department Code,Course Level,GPA,Standing,Completed Credits,...,GPA Student - Subject,Avg. Grade - Taken,Avg. Grade - Students_Subject,Letter Grade,Status,Semester,Theoritical,Practical,Course Credit,ECTS
0,UNI 113,Textual Analysis and Effective Com. I,NaN,1,351,IE,Undergraduate,4.05,Freshman,18,...,4.050000,2.120833,2.703226,A+,Successful,2010 - Fall,3,0,3,5
1,EM 475,Stratejik Yönetim,NaN,4,1227,MBA/NT,Graduate,3.00,Freshman,15,...,3.300000,3.227273,3.227273,B+,Successful,2010 - Fall,3,0,3,5
2,UNI 101,Mathematical Reasoning,NaN,1,344,EE,Undergraduate,4.03,Freshman,18,...,4.033333,2.144444,2.703226,A,Successful,2010 - Fall,3,0,3,5
3,UNI 203,Understanding Science and Technology,NaN,2,344,EE,Undergraduate,4.03,Freshman,18,...,4.033333,3.176000,2.703226,A,Successful,2010 - Fall,3,0,3,5
4,UNI 203,Understanding Science and Technology,NaN,2,240,SOC,Undergraduate,2.62,Freshman,18,...,2.616667,3.176000,2.703226,B,Successful,2010 - Fall,3,0,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44151,PHYS 104L,Physics II - Lab,NaN,1,2001,CS,Undergraduate,2.57,Freshman,21,...,3.000000,2.567082,2.166329,A-,Successful,2014 - Spring,0,2,1,1
44152,EECS 202,Basic Digital Communication with Networking,NaN,2,1313,EE,Undergraduate,2.40,Sophomore,19,...,1.740000,1.980226,2.278289,B-,Successful,2014 - Spring,3,2,4,6
44153,UNI 117,Understanding Society and Culture,NaN,1,2006,EE,Undergraduate,1.49,Freshman,18,...,2.000000,1.923731,2.448116,D,Successful,2014 - Spring,3,0,3,5
44154,EECS 218,Digital Logic Design,NaN,2,1310,EE,Undergraduate,1.86,Sophomore,15,...,0.833333,2.274528,2.278289,F,Unsuccessful,2014 - Spring,3,0,3,5


In [4]:
df.drop([df.columns[0], df.columns[1], df.columns[2], df.columns[4]], inplace=True, axis=1)
df.head()

,Course Year,Department Code,Course Level,GPA,Standing,Completed Credits,Completed ECTS,GPA Student - Subject,Avg. Grade - Taken,Avg. Grade - Students_Subject,Letter Grade,Status,Semester,Theoritical,Practical,Course Credit,ECTS
0,1,IE,Undergraduate,4.05,Freshman,18,30,4.050000,2.120833,2.703226,A+,Successful,2010 - Fall,3,0,3,5
1,4,MBA/NT,Graduate,3.00,Freshman,15,25,3.300000,3.227273,3.227273,B+,Successful,2010 - Fall,3,0,3,5
2,1,EE,Undergraduate,4.03,Freshman,18,30,4.033333,2.144444,2.703226,A,Successful,2010 - Fall,3,0,3,5
3,2,EE,Undergraduate,4.03,Freshman,18,30,4.033333,3.176000,2.703226,A,Successful,2010 - Fall,3,0,3,5
4,2,SOC,Undergraduate,2.62,Freshman,18,30,2.616667,3.176000,2.703226,B,Successful,2010 - Fall,3,0,3,5


In [5]:
df = pd.concat([df, pd.get_dummies(df['Course Year'], prefix='Course Year'), pd.get_dummies(df['Department Code'], prefix='Department Code'), pd.get_dummies(df['Course Level'], prefix='Course Level'), pd.get_dummies(df['Standing'], prefix='Standing'), pd.get_dummies(df['Status'], prefix='Status')], axis=1)
df.drop(['Course Year', 'Department Code', 'Course Level', 'Status', 'Standing'], axis=1, inplace=True)
df

,GPA,Completed Credits,Completed ECTS,GPA Student - Subject,Avg. Grade - Taken,Avg. Grade - Students_Subject,Letter Grade,Semester,Theoritical,Practical,...,Department Code_İHP,Course Level_Graduate,Course Level_Undergraduate,Standing_Freshman,Standing_Junior,Standing_Senior,Standing_Sophomore,Status_Recorded,Status_Successful,Status_Unsuccessful
0,4.05,18,30,4.050000,2.120833,2.703226,A+,2010 - Fall,3,0,...,0,0,1,1,0,0,0,0,1,0
1,3.00,15,25,3.300000,3.227273,3.227273,B+,2010 - Fall,3,0,...,0,1,0,1,0,0,0,0,1,0
2,4.03,18,30,4.033333,2.144444,2.703226,A,2010 - Fall,3,0,...,0,0,1,1,0,0,0,0,1,0
3,4.03,18,30,4.033333,3.176000,2.703226,A,2010 - Fall,3,0,...,0,0,1,1,0,0,0,0,1,0
4,2.62,18,30,2.616667,3.176000,2.703226,B,2010 - Fall,3,0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44151,2.57,21,33,3.000000,2.567082,2.166329,A-,2014 - Spring,0,2,...,0,0,1,1,0,0,0,0,1,0
44152,2.40,19,31,1.740000,1.980226,2.278289,B-,2014 - Spring,3,2,...,0,0,1,0,0,0,1,0,1,0
44153,1.49,18,28,2.000000,1.923731,2.448116,D,2014 - Spring,3,0,...,0,0,1,1,0,0,0,0,1,0
44154,1.86,15,24,0.833333,2.274528,2.278289,F,2014 - Spring,3,0,...,0,0,1,0,0,0,1,0,0,1


In [6]:
columns = df.columns
columns

Index(['GPA', 'Completed Credits', 'Completed ECTS', 'GPA Student - Subject',
       'Avg. Grade - Taken', 'Avg. Grade - Students_Subject', 'Letter Grade',
       'Semester', 'Theoritical', 'Practical', 'Course Credit', 'ECTS',
       'Course Year_1', 'Course Year_2', 'Course Year_3', 'Course Year_4',
       'Department Code_CS', 'Department Code_CTV', 'Department Code_CULT/T',
       'Department Code_DSE/T', 'Department Code_ECE/NT',
       'Department Code_ECE/T', 'Department Code_EE', 'Department Code_HIST',
       'Department Code_HIST PhD', 'Department Code_HIST/T',
       'Department Code_HUK', 'Department Code_IE', 'Department Code_ISE/NT',
       'Department Code_ISE/T', 'Department Code_ISS', 'Department Code_ITM',
       'Department Code_LIT', 'Department Code_MBA/NT', 'Department Code_MGT',
       'Department Code_MTS/T', 'Department Code_PHIL', 'Department Code_POLS',
       'Department Code_POLS/NT', 'Department Code_POLS/T',
       'Department Code_PSY', 'Department Code_

In [7]:
le = LabelEncoder()
le.fit(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F'])

LabelEncoder()

In [8]:
def get_train_data(df, train_sem, columns):
    dataFrame = pd.DataFrame(columns=columns)
    for sem in train_sem:
        dataFrame = pd.concat([dataFrame, df[df.iloc[:, 7] == sem]], ignore_index=True)
    for idx in dataFrame.index:
        semester = dataFrame.iloc[idx, 7]
        dataFrame.iloc[idx, 7] = semester.split()[2].strip()
    
    X_train = dataFrame.drop('Semester', axis=1)
    y_train = le.transform(X_train.pop('Letter Grade'))
    return X_train, y_train

In [9]:
def normalize_X(X):
    cols = X.columns
    sc = StandardScaler()
    X = sc.fit_transform(X)
    return pd.DataFrame(X, columns=cols)

In [18]:
sorted_semesters = sorted(set(df.iloc[:, 7]))
training_sem = sorted_semesters[:len(sorted_semesters)-1]

In [19]:
X_train, y_train = get_train_data(df, training_sem, columns)

In [21]:
X_train = normalize_X(X_train)

In [22]:
def tune_params(model, param_grid, X_train, y_train):
    gridsearch_CV = GridSearchCV(model, param_grid, cv=10, n_jobs=-1)
    gridsearch_CV.fit(X_train, y_train)
    return gridsearch_CV.best_params_

In [27]:
adaboost_param_grid = {'n_estimators': [20, 50, 100, 200, 1000], 'loss': ['linear', 'square', 'exponential']}
best_ab = tune_params(AdaBoostRegressor(), adaboost_param_grid, X_train, y_train)
adaboost_params = {}
with open('adaboost_tuned_params.json', 'w') as file:
    adaboost_params['tuned_adaboost'] = best_ab
    json.dump(adaboost_params, file)

In [28]:
bayesian_param_grid = {'n_iter': [300, 800, 1500], 'alpha_1': [1e-4, 1e-6, 1e-8], 'alpha_2': [1e-4, 1e-6, 1e-8], 'lambda_1': [1e-4, 1e-6, 1e-8], 'lambda_2': [1e-4, 1e-6, 1e-8]}
best_br = tune_params(BayesianRidge(), bayesian_param_grid, X_train, y_train)
bayesian_params = {}
with open('bayesian_tuned_params.json', 'w') as file:
    bayesian_params['tuned_bayesian'] = best_br
    json.dump(bayesian_params, file)

In [30]:
ridge_param_grid = {'alpha': [0.01, 0.2, 1.0, 10.0, 20.0, 50.0, 100.0, 500.0]}
best_r = tune_params(Ridge(), ridge_param_grid, X_train, y_train)
ridge_params = {}
with open('ridge_tuned_params.json', 'w') as file:
    ridge_params['tuned_ridge'] = best_r
    json.dump(ridge_params, file)